In [52]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from lightgbm.callback import early_stopping, log_evaluation
from sklearn.model_selection import StratifiedKFold
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
import warnings

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

submission_files = []

import os
for dirname, _, filenames in os.walk('/kaggle/input/tabular-playground-series-nov-2022/submission_files/'):
    for filename in filenames:
        submission_files.append(os.path.join(filename).replace('.csv',''))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [53]:
train_labels = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2022/train_labels.csv",index_col="id",squeeze=True)
sample_submission = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2022/sample_submission.csv",index_col='id')

In [54]:

good_roc_auc = ['0.6426110725.csv', '0.6453694232.csv', '0.6454375051.csv', '0.6475577136.csv', '0.6483090984.csv', '0.6485743466.csv', '0.6492627054.csv', '0.6493925862.csv', '0.6495194746.csv', '0.6495843938.csv', '0.6498444213.csv', '0.6498855044.csv', '0.6500630905.csv', '0.6500863890.csv', '0.6501070159.csv', '0.6501134385.csv', '0.6503186151.csv', '0.6507043652.csv', '0.6510016540.csv', '0.6513124137.csv', '0.6515649417.csv', '0.6517195011.csv', '0.6518079470.csv', '0.6522976597.csv', '0.6523331629.csv', '0.6524028654.csv', '0.6532083105.csv', '0.6538530574.csv', '0.6541278746.csv', '0.6544952482.csv', '0.6552358052.csv', '0.6557547476.csv', '0.6560480516.csv', '0.6563223387.csv', '0.6564147356.csv', '0.6564517315.csv', '0.6565234650.csv', '0.6570599977.csv', '0.6572293651.csv', '0.6581980033.csv', '0.6582025555.csv', '0.6583138987.csv', '0.6585612585.csv', '0.6585799099.csv', '0.6585907664.csv', '0.6587034723.csv', '0.6587474948.csv', '0.6587775987.csv', '0.6588664218.csv', '0.6589328974.csv', '0.6589649349.csv', '0.6589751776.csv', '0.6590005376.csv', '0.6592172566.csv', '0.6592570649.csv', '0.6593057142.csv', '0.6593521643.csv', '0.6594027820.csv', '0.6596263624.csv', '0.6596476913.csv', '0.6597075436.csv', '0.6597363850.csv', '0.6597756381.csv', '0.6598913657.csv', '0.6599082809.csv', '0.6599155304.csv', '0.6599181044.csv', '0.6601412175.csv', '0.6601920939.csv', '0.6603097589.csv', '0.6603536662.csv', '0.6603564013.csv', '0.6603869261.csv', '0.6603949762.csv', '0.6605086779.csv', '0.6606298703.csv', '0.6607092002.csv', '0.6611595537.csv', '0.6615508013.csv', '0.6618187844.csv', '0.6620129196.csv', '0.6620201058.csv', '0.6620495604.csv', '0.6620966154.csv', '0.6633472767.csv', '0.6635501461.csv', '0.6639347225.csv', '0.6640531222.csv', '0.6641563772.csv', '0.6643704680.csv', '0.6648661697.csv', '0.6662736977.csv', '0.6681069236.csv', '0.6702783631.csv', '0.6707095794.csv', '0.6708614395.csv', '0.6710450756.csv', '0.6778730537.csv', '0.6924123007.csv']


good_roc_auc = list(map(lambda x : x.replace('.csv',''),good_roc_auc)) #found in another notebook

In [55]:
path = '/kaggle/input/tabular-playground-series-nov-2022/submission_files/'
df = pd.DataFrame()
for idx,number_name in enumerate(submission_files):
    if float(number_name)<0.693 and number_name in good_roc_auc:
        name = 'sub_{}'.format(idx)
        df[name] = pd.read_csv(path + number_name +'.csv',skipfooter=len(train_labels),engine='python',squeeze=True,usecols=['pred'])

In [56]:
df

,sub_29,sub_35,sub_93,sub_94,sub_137,sub_167,sub_177,sub_185,sub_216,sub_249,...,sub_4386,sub_4465,sub_4508,sub_4570,sub_4604,sub_4652,sub_4673,sub_4778,sub_4929,sub_4934
0,0.971786,0.880978,0.902283,0.922075,0.921742,0.904482,0.956259,0.843527,0.943179,0.930242,...,0.929678,0.893934,0.941171,0.945280,0.890427,0.941618,0.934658,0.916432,0.879518,0.850759
1,0.692161,0.666823,0.577745,0.604563,0.619580,0.652571,0.642106,0.625227,0.680749,0.629101,...,0.642914,0.609602,0.599508,0.652881,0.596047,0.628834,0.619196,0.627445,0.661202,0.669435
2,0.816399,0.802251,0.844702,0.814117,0.832032,0.807416,0.795874,0.816146,0.809795,0.809876,...,0.820717,0.848504,0.828846,0.799195,0.825430,0.811948,0.843441,0.837226,0.825571,0.829589
3,0.814499,0.787668,0.795583,0.817518,0.808295,0.795489,0.814228,0.772266,0.855050,0.842175,...,0.818054,0.758500,0.840623,0.800496,0.797684,0.832319,0.789155,0.788663,0.738148,0.792266
4,0.950065,0.946063,0.952976,0.951825,0.942013,0.948601,0.945268,0.934874,0.940145,0.964983,...,0.947651,0.954260,0.958420,0.950246,0.950310,0.958250,0.944987,0.957015,0.930563,0.959891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.930625,0.902832,0.935450,0.884759,0.890019,0.883887,0.945354,0.901941,0.912217,0.889872,...,0.854774,0.916351,0.918071,0.938664,0.926305,0.898133,0.894353,0.936260,0.895371,0.952819
19996,0.723180,0.589014,0.653084,0.650025,0.627651,0.636267,0.660644,0.659820,0.611244,0.628845,...,0.646381,0.727248,0.632751,0.678112,0.774145,0.675560,0.704070,0.619173,0.604725,0.608933
19997,0.936813,0.954243,0.921845,0.937934,0.940623,0.919551,0.937326,0.927416,0.941886,0.959954,...,0.958117,0.922556,0.943099,0.936229,0.909828,0.940854,0.940089,0.920169,0.918245,0.956898
19998,0.687782,0.696916,0.642598,0.691664,0.705262,0.720336,0.685690,0.735075,0.742624,0.655938,...,0.707196,0.731064,0.680868,0.684782,0.666499,0.688920,0.700207,0.632268,0.751161,0.673240


In [57]:
 X_train, X_test, y_train, y_test = train_test_split(df,train_labels,test_size=0.2,shuffle=True,random_state=42)

In [58]:
df2 = df.copy()
df2["result"] = train_labels

In [59]:
warnings.filterwarnings("ignore")
k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fit_params = {
             "eval_metric" : 'binary_logloss', 
             "eval_set" : [(X_test, y_test)],
    'callbacks':[early_stopping(100), log_evaluation(100)],
             'verbose':0}

param_test = {
              'learning_rate' : [0.01, 0.02, 0.03, 0.04, 0.05, 0.08, 0.1, 0.2, 0.3, 0.4],
              'n_estimators' : [100, 200, 300, 400, 500, 600, 800, 1000, 1500, 2000],
              'num_leaves': sp_randint(6, 50, 100), 
              'min_child_samples': sp_randint(100, 500), 
              'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
              'subsample': sp_uniform(loc=0.2, scale=0.8), 
              'max_depth': [-1, 1, 2, 3, 4, 5, 6, 7],
              'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
              'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
              'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100],
             }



lgbm_rs = LGBMClassifier(metric= 'binary_logloss', 
                         random_state=42, 
                         silent=True, 
                         n_jobs=-1,
                         zero_as_missing=True
                        )

random_search = RandomizedSearchCV(
    estimator=lgbm_rs, 
    param_distributions=param_test,     
    scoring='neg_log_loss',
    cv=k_fold,
    refit=True,
    random_state=42,
    n_iter=20
)

random_search.fit(X_train, y_train, **fit_params)
opt_parameters =  random_search.best_params_

warnings.filterwarnings("always")

Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.525622
Early stopping, best iteration is:
[20]	valid_0's binary_logloss: 0.521821
[100]	valid_0's binary_logloss: 0.52693
Early stopping, best iteration is:
[20]	valid_0's binary_logloss: 0.521821
[100]	valid_0's binary_logloss: 0.525039
Early stopping, best iteration is:
[30]	valid_0's binary_logloss: 0.521658
[100]	valid_0's binary_logloss: 0.525679
Early stopping, best iteration is:
[30]	valid_0's binary_logloss: 0.521658
[100]	valid_0's binary_logloss: 0.523379
Early stopping, best iteration is:
[30]	valid_0's binary_logloss: 0.521502
[100]	valid_0's binary_logloss: 0.523696
Early stopping, best iteration is:
[30]	valid_0's binary_logloss: 0.521502
[100]	valid_0's binary_logloss: 0.52386
Early stopping, best iteration is:
[30]	valid_0's binary_logloss: 0.521502
[100]	valid_0's binary_logloss: 0.523237
Early stopping, best iteration is:
[30]	valid_0's binary_logloss: 0.521502
[100]	valid_

In [60]:
parameters = opt_parameters
parameters['device']='cpu'
parameters['randomstate']=42
parameters

{'colsample_bytree': 0.4004672595046086,
 'learning_rate': 0.05,
 'max_depth': -1,
 'min_child_samples': 413,
 'min_child_weight': 10.0,
 'n_estimators': 1500,
 'num_leaves': 132,
 'reg_alpha': 1,
 'reg_lambda': 5,
 'subsample': 0.6113875507308892,
 'device': 'cpu',
 'randomstate': 42}

In [61]:
model = LGBMClassifier(**parameters)

model.fit(df,train_labels)

LGBMClassifier(colsample_bytree=0.4004672595046086, device='cpu',
               learning_rate=0.05, min_child_samples=413, min_child_weight=10.0,
               n_estimators=1500, num_leaves=132, randomstate=42, reg_alpha=1,
               reg_lambda=5, subsample=0.6113875507308892)

In [62]:
df2 = pd.DataFrame()
for idx,number_name in enumerate(submission_files):
    name = 'sub_{}'.format(idx)
    if name in df.columns:
        df2[name] = pd.read_csv(path + number_name +'.csv',skiprows=range(1,len(train_labels)+1),engine='python',squeeze=True,usecols=['pred'])

In [63]:
df2

,sub_29,sub_35,sub_93,sub_94,sub_137,sub_167,sub_177,sub_185,sub_216,sub_249,...,sub_4386,sub_4465,sub_4508,sub_4570,sub_4604,sub_4652,sub_4673,sub_4778,sub_4929,sub_4934
0,0.736929,0.831459,0.854607,0.826338,0.841679,0.857580,0.831996,0.747944,0.865010,0.823650,...,0.834630,0.841519,0.825627,0.848975,0.846603,0.837002,0.845408,0.833249,0.804686,0.831221
1,0.854790,0.875145,0.855292,0.891609,0.891585,0.856955,0.852473,0.870693,0.886616,0.874448,...,0.867439,0.896247,0.907269,0.841868,0.856345,0.894160,0.830832,0.852864,0.871109,0.833086
2,0.516469,0.460372,0.480734,0.507006,0.499185,0.515437,0.497653,0.565207,0.455009,0.422112,...,0.395522,0.441567,0.484380,0.511539,0.485252,0.453071,0.514929,0.446072,0.605487,0.528836
3,0.714897,0.599066,0.551250,0.661131,0.651588,0.616005,0.610484,0.785767,0.567715,0.595485,...,0.668585,0.653498,0.697884,0.618288,0.665879,0.638065,0.673959,0.577041,0.801238,0.604100
4,0.095820,0.176743,0.475863,0.153227,0.108488,0.309533,0.359110,0.498804,0.307581,0.154978,...,0.301587,0.155776,0.179125,0.401827,0.509312,0.212926,0.230148,0.723775,0.050128,0.467353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.487387,0.441519,0.492116,0.480533,0.471065,0.429488,0.518338,0.474897,0.463373,0.427306,...,0.416535,0.465082,0.600824,0.509701,0.582944,0.456128,0.481499,0.487065,0.507386,0.449451
19996,0.594491,0.604038,0.518062,0.559754,0.570755,0.581373,0.663337,0.614598,0.604600,0.596430,...,0.606266,0.605378,0.570469,0.648563,0.642517,0.567946,0.640908,0.571799,0.629882,0.606149
19997,0.724796,0.783718,0.770987,0.760474,0.759494,0.750866,0.748020,0.771637,0.796580,0.728781,...,0.772380,0.743673,0.814989,0.760732,0.693231,0.745517,0.744612,0.746743,0.758835,0.785386
19998,0.719688,0.727606,0.745158,0.727397,0.696729,0.779370,0.735079,0.798413,0.725119,0.743086,...,0.646837,0.766096,0.738108,0.757785,0.816824,0.715426,0.772877,0.773590,0.811537,0.726399


In [64]:
test = df2.sample(1000)

In [65]:
#df2.drop(columns = "result",inplace=True)

In [66]:
somme = list(model.predict_proba(df2)[:,1])

In [67]:
sample_submission['pred'] = somme

In [68]:
sample_submission.to_csv('test.csv')

In [69]:
sample_submission

,pred
id,
20000,0.732800
20001,0.587376
20002,0.040405
20003,0.656748
20004,0.017564
...,...
39995,0.015935
39996,0.299774
39997,0.427178


In [70]:
df3= pd.DataFrame()
for idx,number_name in enumerate(submission_files):
    name = 'sub_{}'.format(idx)
    if name in df.columns:
        df3[name] = pd.read_csv(path + number_name +'.csv',squeeze=True,usecols=['pred'])

In [71]:
df3.to_csv("df.csv")